In [29]:
from __future__ import print_function
import requests
import json
import time
import pandas as pd

URL = 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2020-08-02&end_date=2020-08-09&api_key=cpesbQTFO30gsOz47Enfzn87YXLRdfjxClYjaMdM'
response = requests.get(URL)

if(response.ok):
    jData = json.loads(response.content) #all json file data
    #print(jData)
    
    lst = []
    
    n = []
    mag = []
    min_diam = []
    max_diam = [] 
    hazard = []
    appr = []
    appr_time = []
    vel = []
    miss = []
    final_lst = []
    
    neo_obj_date = jData['near_earth_objects'].keys() 
    for i in neo_obj_date:
        lst.append(i)
        #print(i) #the celestial objects that have their approach closest to earth on this date
    
        data_for_date = jData['near_earth_objects'][i]
        #print(data_for_date)
        
        for j in data_for_date:
            name = j['name'] #names of all objects at specific date
            #print(name)
            n.append(name)
    
            abs_magnitude = round(j['absolute_magnitude_h'], 2)
            #print(abs_magnitude)
            mag.append(abs_magnitude)
            
            est_diameter_min = str(round(j['estimated_diameter']['meters']['estimated_diameter_min'], 2)) + ' m'
            #print(est_diameter_min)
            min_diam.append(est_diameter_min)
                
            est_diameter_max = str(round(j['estimated_diameter']['meters']['estimated_diameter_max'], 2)) + ' m'
            #print(est_diameter_max)
            max_diam.append(est_diameter_max)
            
            is_hazardous = j['is_potentially_hazardous_asteroid']
            #print(is_hazardous)
            hazard.append(is_hazardous)
            
            close_approach_data1 = j['close_approach_data']
            #print(close_approach_data1)
            for k in close_approach_data1:
                if k['close_approach_date_full'] is not None: #displays date with time
                    close_approach_date1 = k['close_approach_date_full']
                    sp = close_approach_date1.split(" ")
                    hours, minutes = sp[1].split(":") #military to standard time conversion
                    
                    h = int(hours)
                    mins = int(minutes)
                    val = " A.M."
                    
                    if h > 12:
                        val = " P.M."
                        h -= 12
                        #print(("%02d:%02d" + val) % (h, mins))
                        out = ("%02d:%02d" + val) % (h, mins)
                    elif h == 0:
                        h += 12
                        #print(("%02d:%02d" + val) % (h, mins))
                        out = ("%02d:%02d" + val) % (h, mins)
                    else:
                        #print(("%02d:%02d" + val) % (h, mins))
                        out = ("%02d:%02d" + val) % (h, mins)
                    
                    #close_approach_date = sp[0] + " " + out
                    close_approach_date_time = out
                    appr.append(close_approach_date)
                    appr_time.append(close_approach_date_time)
                    
                else: #displays date without time
                    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
                    month_nums = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
                    date = k['close_approach_date'].split("-")
                    for l in month_nums:
                        if date[1] == l:
                            #print(month_names[int(l)-1])
                            close_approach_date = date[0] + '-' + month_names[int(l)-1] + '-' + date[2]
                            appr.append(close_approach_date)
                            appr_time.append('N/A')
                        
            #print(close_approach_date)
            
                relative_velocity = str(round(float(k['relative_velocity']['kilometers_per_second']), 2)) + ' km/s'
                #print(relative_velocity)
                vel.append(relative_velocity)
                
                miss_distance = str(round(float(k['miss_distance']['lunar']), 1)) + ' LD'
                #print(miss_distance)
                miss.append(miss_distance)
    
    f_df = pd.DataFrame()
    f_df['Name'] = n
    f_df['Absolute Magnitude'] = mag
    f_df['Minimum Estimated Diameter'] = min_diam
    f_df['Maximum Estimated Diameter'] = max_diam
    f_df['Potentially Hazardous'] = hazard
    f_df['Closest Approach Date'] = appr
    f_df['Time of Closest Approach Date'] = appr_time
    f_df['Relative Velocity'] = vel
    f_df['Miss Distance'] = miss
    #f_df = f_df.drop(columns = [0])
    f_df.to_excel('NASA API Web Scraping.xlsx', sheet_name = 'NASA API Web Scraping')
    #print(f_df)
    
else:
    response.raise_for_status()
        

                     Name  Absolute Magnitude Minimum Estimated Diameter  \
0              (2019 AR9)               21.70                   121.49 m   
1               (2011 MF)               21.90                    110.8 m   
2              (2020 MO4)               21.64                   124.96 m   
3   9162 Kwiila (1987 OA)               17.80                   732.07 m   
4              (2006 EC1)               18.50                   530.34 m   
..                    ...                 ...                        ...   
86            (2015 FM33)               24.60                    31.96 m   
87              (2018 YH)               21.80                   116.03 m   
88            (2020 BW12)               26.24                    15.02 m   
89             (2020 MC4)               21.41                   138.85 m   
90             (2020 NJ1)               22.65                    78.55 m   

   Maximum Estimated Diameter  Potentially Hazardous Closest Approach Date  \
0        